# Where should I go? Finding the optimal neighborhood in Bonn to move to from Munich

## Author: Laila Linke

## 1. Data Scraping and Cleaning

In [1]:
# Import packages
import folium #To plot maps
import geocoder # To get coordinates from addresses
#!pip install geopandas
import geopandas as gpd # To deal with geojson files
import requests # For https requests
import pandas as pd

In [2]:
# Function that gives out coordinates for a given address
# Uses ArcGIS (World geocoding service) instead of Google API
def getCoordinates(address):   
    coords=None
    while(coords==None): # Repeat call, until return is valid coordinate
        g=geocoder.arcgis(address)
        coords=g.latlng
    return coords

### 1.1. Get positions of Bonn Neighborhoods

In [3]:
bonn=r'Bonn.json' # Name of geojson file containing Bonn neighborhoods
coord_bonn=getCoordinates("Bonn") # Get coordinates of Bonns City centre from ArcGIS
coord_bonn

[50.73242000000005, 7.101860000000045]

In [4]:
# Visualize Bonns neighborhoods with Folium
map_bonn=folium.Map(location=coord_bonn, zoom_start=13)
map_bonn.choropleth(
    geo_data=bonn,
    fill_opacity=0.2,
)
map_bonn

In [5]:
# Read in geojson
neighborhoods=gpd.read_file(bonn)
# Compute centres of each neighborhood with geopandas function
neighborhoods['longitude']=neighborhoods.centroid.x
neighborhoods['latitude']=neighborhoods.centroid.y

#Drop unnecessary columns: geopandas ID, neighborhood number, district number and district name
neighborhoods.drop(['gid', 'ortsteil', 'bezirk', 'bezirk_bez'], inplace=True, axis=1)

#Rename column with neighborhood name
neighborhoods.rename(columns={"ortsteil_bez": "name"}, inplace=True)
neighborhoods.head()

<ipython-input-5-87e9623d687b>:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  neighborhoods['longitude']=neighborhoods.centroid.x
<ipython-input-5-87e9623d687b>:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  neighborhoods['latitude']=neighborhoods.centroid.y


,name,geometry,longitude,latitude
0,Auerberg,"POLYGON ((7.06486 50.75323, 7.06502 50.75339, ...",7.070968,50.755913
1,Bonn-Castell,"POLYGON ((7.09779 50.75559, 7.09781 50.75560, ...",7.097325,50.747996
2,Bonn-Zentrum,"POLYGON ((7.10191 50.73247, 7.10163 50.73225, ...",7.101694,50.736540
3,Buschdorf,"POLYGON ((7.06502 50.75339, 7.06486 50.75323, ...",7.053905,50.757436
4,Dottendorf,"POLYGON ((7.10961 50.70893, 7.10961 50.70890, ...",7.115320,50.703584


In [6]:
# Add neighborhood centres with labels to map of Bonn

for lat, lon, name in zip(neighborhoods['latitude'], neighborhoods['longitude'], neighborhoods['name']):
    label=folium.Popup(name, parse_html=True) # Add label with neighborhood name to each circle
    folium.CircleMarker([lat, lon], popup=label, radius=1).add_to(map_bonn)
map_bonn

### 1.2. Get position of current home in Munich

In [7]:
address="Christoph-Probst-Straße 10, 80805 München" # From https://www.studentenwerk-muenchen.de/wohnen/wohnanlagen/muenchen/muenchen-nord/wohnanlage-studentenstadt-freimann/
coord_munich=getCoordinates(address)
print("The current home in Munich is at ({},{})".format(coord_munich[0], coord_munich[1]))

The current home in Munich is at (48.18426001732436,11.609809962850903)


### 1.3. Get venues in each neighborhood in Bonn from FourSquare

In [8]:
# Foursquare credentials
CLIENT_ID = 'MSKA2MHIDITZYMFO2A42PE2SXYI2PWPXR24DBFLI43HN5Y14' # your Foursquare ID
CLIENT_SECRET = '25VATPCXE3W5SCML34NOTVM2BAHHQISAOJRE2FRV5JSKJJYN' # your Foursquare Secret
VERSION = '20201217' # Foursquare API version
LIMIT = 300 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MSKA2MHIDITZYMFO2A42PE2SXYI2PWPXR24DBFLI43HN5Y14
CLIENT_SECRET:25VATPCXE3W5SCML34NOTVM2BAHHQISAOJRE2FRV5JSKJJYN


In [9]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print('Reading out venues in {} at ({},{})'.format(name, lat, lng))
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&intent=checkin'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()#["response"]['groups'][0]['items']
        print(results)
        #return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',
                             'Venue Category']
    
    return(nearby_venues)

In [10]:
# Get venues in Bonn
venues_Bonn=getNearbyVenues(neighborhoods['name'], neighborhoods['longitude'], neighborhoods['latitude'])
venues_Bonn

Reading out venues in Auerberg at (7.070967503548737,50.755913056210495)
{'meta': {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5fdb8c0120255f5eb092abac'}, 'response': {}}


TypeError: string indices must be integers

### 1.4. Get venues near current home in Munich

In [ ]:
venues_Munich=getNearbyVenues(['Munich'], [coord_munich[0]], [coord_munich[1]])
venues_Munich